<a href="https://colab.research.google.com/github/MahuL88/SpecialSkill/blob/main/Car_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Day 1

## Bagian 1

* Clone YOLOv5 dan install dependensi

In [ ]:
# Melakukan perintah clone kode YOLOV5 dari github repo ultralytics ke collab melalui terimnal
!git clone https://github.com/ultralytics/yolov5
# Pindah direktori dari direktori/folder utama collab ke yolov5 dengan bantuan magic command
%cd yolov5
# Menginstall semua library/pustaka yang dibutuhkan oleh YOLOv5 di file requirement.txt melalui terminal.
!pip install -r requirements.txt

# Menginstall library deep sort untuk tracking(memberi ID) melalui terminal
!pip install deep_sort_realtime

Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.53 MiB | 12.73 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━

* Untuk contoh file videonya bisa diakses/didownload di sini : [teks link](https://drive.google.com/file/d/1_fvKmLqrQlRKEIqrdfixIRfcV5dkU-cT/view?usp=sharing)

In [ ]:
from google.colab import files # Impor modul files dari library collab.

uploaded = files.upload() # Mendefinisikan variabel uploaded agar dapat mengunggah file

Saving car_video.mp4 to car_video.mp4


* Sebelumnya saya membandingkan model YOLOv5(s,m,l,x) untuk melihat mana yang akurat melakukan bounding box dan proses training yang tidak lama dari sebuah gambar. akhirnya, saya memilih model YOLOv5m

In [ ]:
# Impor torch dari framework PyTorch, impor modul deepsort, dan pustaka OpenCV(untuk mengolah model dan video)
import torch
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort

# Membuat variabel model dimana torch mendownload model versi "m" yang sudah dilatih sebelumnya (pretrained).
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
# Mengecek apakah GPU aktif di Google Colab.
# Untuk Mengaktifkannya, di fitur runtime, ubah jenis runtime dengan runtume GPU (akan mengembalikan bool True jika aktif)
print(torch.cuda.is_available())

/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-5-29 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

100%|██████████| 40.8M/40.8M [00:00<00:00, 117MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


True


#Day 2

In [ ]:
# Inisialisasi Deep SORT tracker yg akan menyimpan objek yg hilang selama 30 frame
tracker = DeepSort(max_age=30)

# Variabel video_path yang akan menyimpan nama file video
video_path = 'car_video.mp4'

# membuka/membaca video tersebut yang akan memproses frame demi frame, lalu menyimpannya ke variabel cap
cap = cv2.VideoCapture(video_path)

# Variabel output_path utk simpan hasil file yang sudah di bounding box nantinya
output_path = 'output.avi'

# Variabel Four Character Code utk memilih format kompresi video outputnya dengan jenis kompresi XVID
fourcc = cv2.VideoWriter_fourcc(*'XVID')

# Mengambil/membaca lebarnya frame dengan satuan pixel bertipe integer
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# Mengambil/membaca Tingginya frame dengan satuan pixel bertipe integer
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Membuat objek VideoWriter untuk menulis frame ke dalam file video
# Dengan parameter, nama file output, format video disimpan, frame per second, dan ukuran frame video output
out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))

# Perulangan dengan kondisi selama video masih bisa dibuka, baca frame dan kembalikan status True
# Jika status sudah False, artinya video sudah selesai dan loop berhenti
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Model mendeteksi objek " di dalam frame dan disimpan ke variabel results
    results = model(frame)
    # mengembalikan hasil deteksi dalam bentuk tabel(pandas df) yang memiliki kolom class dan nama
    detections = results.pandas().xyxy[0]

    # menyiapkan list kosong untuk menyimpan data yg akan ditracker
    dets_for_sort = []

    # Melakukan Bounding Box dan filter hanya untuk class saja
    for _, row in detections.iterrows():  # baca hasil deteksi baris demi baris
        if row['name'] == 'car':          # filter hanya utk class car

            # Ambil koordinat kotak dari setiap baris yg sudah difilter
            x1, y1, x2, y2 = map(int, [row['xmin'], row['ymin'], row['xmax'], row['ymax']])

            # Menghitung lebar (w) dan tinggi (h) kotak mobil
            w, h = x2 - x1, y2 - y1

            # Menghitung tingkat keyakinan model mendeteksi mobil
            conf = row['confidence']

            # masukkan data ke list kosong sebelumnya dalam format [x, y, w, h], confidence, dan label
            dets_for_sort.append(([x1, y1, x2 - x1, y2 - y1], conf, 'car'))

    # Update tracker dengan mengirim hasil deteksi, diproses tracker, lalu tiap mobil diberikan ID unik
    tracks = tracker.update_tracks(dets_for_sort, frame=frame)

    # Perulangan untuk melihat mobil/objek yang ada di tracks
    for track in tracks:
        # Jika objek masih belum terkonfirmasi (track tidak valid), lewatkan
        if not track.is_confirmed():
            continue

        # Mengambil ID unik untuk setiap mobil/objek.
        track_id = track.track_id

        # Membaca/mengambil koordinat kotak(bounding box) dalam format left, top, right, and bottom(ltrb)
        ltrb = track.to_ltrb()

        # Ubah semua elemen di ltrb jadi integer agar mudah dipakai OpenCV utk menggambar boxnya
        x1, y1, x2, y2 = map(int, ltrb)

        # Menggambar kotak biru(kode 255, 0, 0) di sekeliling class(mobil)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255,0,0), 2)

        # Menulis text di atas kotak untuk ID object/mobil
        # dengan warna text hijau dan posisi text 10 piksel di atas kotak di sebelah kiri atas.
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    # simpan framenya(yg sudah di kotakin) ke variabel video_path
    out.write(frame)

# Menutup file video input
cap.release()
# Menutup file video output
out.release()

Output streaming akan dipotong hingga 5000 baris terakhir.
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5

In [ ]:
# Mengunduh file output hasil yang telah di bounding box ke perangkat lokal(komputer)
files.download('/content/yolov5/output.avi')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Ringkasan

Tracking mobil ini mendeteksi mobil menggunakan model YOLOv5m yang akan melihat dan memberi kotak pembatas (bounding box) pada setiap mobil di tiap frame video. lalu, hasilnya diteruskan ke Deep Sort untuk melacak dan memberikan ID unik untuk setiap object(dalam kasus ini mobil), sehingga mobil yang sama bisa dikenali secara konsisten sepanjang video secara real-time.